# Defining Table Metadata with the ORM

Ketika menggunakan ORM, proses dimana kita mendeklarasikan metadata Table biasanya dikombinasikan dengan proses pendeklarasian <b>mapped class</b>. Mapped class ini adalah sebuah python class yang akan dibuat yang akan memiliki atribut  yang akan dihubungkan ke kolom-kolom pada sebuah tabel pada database.

Saat menggunakan ORM koleksi MetaData tetap ada, namun itu adal di dalam objek ORM yang dikenal sebagai <b>registry</b>

In [1]:
from sqlalchemy.orm import registry
mapper_registry = registry()

In [2]:
mapper_registry.metadata

MetaData()

Kali ini kita akan mendeklarasikan objek Table secara tidak langsung melalui mapped class. Dalam pendekatan plaing umum, setiap mapped class diturunkan dari sebuah base class yaitu <b>declarative base</b>. Kita bisa mendapatkan declarative base dengan menggunakan metode <b>generate_base()</b> dari objek registry:

Base = mapper_registry.generate_base()

atau dengan cara yang lebih singkat yaitu

from sqlalchemy.orm import declarative_base <br>
Base = declarative_base()

In [3]:
# from sqlalchemy.orm import declarative_base
# Base = declarative_base()

Base = mapper_registry.generate_base()

Objek Base di atas adalah sebuah python class yang akan berfungsi sebagai base class untuk ORM mapped class yang akan kita deklarasikan. Sekarang kita dapat mendefinisikan ORM mapped class untuk tabel users dan email seperti berikut:

In [4]:
from sqlalchemy import Column, Integer, String, ForeignKey
from sqlalchemy.orm import relationship

class Users(Base):
    __tablename__ = "users"
    
    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    fullname = Column(String)
    
    email_address = relationship("Email", back_populates="user")
    
    def __repr__(self):
        return f'User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})'
    
class Email(Base):
    __tablename__ = "email"
    
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey("users.id"))
    
    user = relationship("Users", back_populates="email_address")
    
    def __repr__(self):
        return f'Email(id={self.id!r}, email_address={self.email_address!r})'

class Users dan Email sekarang adalah mapped class dan tersedia untuk digunakan dalam ORM dan operasi kueri.

In [5]:
Users.__table__

Table('users', MetaData(), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(length=30), table=<users>), Column('fullname', String(), table=<users>), schema=None)

In [6]:
Email.__table__

Table('email', MetaData(), Column('id', Integer(), table=<email>, primary_key=True, nullable=False), Column('email_address', String(), table=<email>, nullable=False), Column('user_id', Integer(), ForeignKey('users.id'), table=<email>), schema=None)

In [7]:
Users.__tablename__

'users'

In [8]:
Users

__main__.Users

In [9]:
muhlisadiwiguna = Users(name="Muhlis", fullname="Muhlis Adiwiguna")

In [10]:
muhlisadiwiguna

User(id=None, name='Muhlis', fullname='Muhlis Adiwiguna')

## Emmiting DDL to the Database

Emmiting menggunakan mode ORM tidak jauh berbeda dengan menggunakan mode core yaitu sebagai berikut:

mapper_registry.metadata.create_all(engine)

atau

Base.metadata.create_all(engine)

In [11]:
from sqlalchemy import create_engine
engine = create_engine("sqlite+pysqlite:///example2.db", future=True)

In [12]:
# cara 1
# mapper_registry.metadata.create_all(engine)

# cara 2
Base.metadata.create_all(engine)